In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pandas as pd

jar_files = [
    "/util/clickhouse-jdbc-0.3.2-all.jar"
]

# Initialize Spark session with JARs
spark = SparkSession.builder \
    .appName("combina_shapes") \
    .master("spark://spark-master:7077") \
    .config("spark.jars", ",".join(jar_files)) \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.access.key","admin") \
    .config("spark.hadoop.fs.s3a.secret.key","minioadmin") \
    .config("spark.hadoop.fs.s3a.endpoint","http://minio:9000") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()


url = "jdbc:ch://clickhouse:8123/cameraaberta"
user = "admin" 
password = "admin"  
driver = "com.clickhouse.jdbc.ClickHouseDriver"

display(spark)

In [2]:
# df_previsao = pd.read_json(f'./previsao.json', ignore)
# print(df_previsao.shape)
# df_previsao.head()

In [3]:
df_stops = spark.read.option("inferSchema", True).option("header", "true") \
    .csv(f"s3a://raw/stops.txt").toPandas()
# show data
df_stops.head(10)

,stop_id,stop_name,stop_desc,stop_lat,stop_lon
0,18848,Clínicas,None,-23.554022,-46.671108
1,18849,Vila Madalena,None,-23.546498,-46.691141
2,18850,Consolação,None,-23.558094,-46.660205
3,18851,Conceição,None,-23.635039,-46.641239
4,18852,Jabaquara,None,-23.646033,-46.641028
5,18853,São Judas,None,-23.625882,-46.640936
6,18854,Saúde,None,-23.618245,-46.639139
7,18855,Praça Da Árvore,None,-23.610583,-46.637918
8,18856,Santa Cruz,None,-23.598541,-46.636638
9,18857,Vila Mariana,None,-23.589359,-46.634677


In [4]:
df_trips = spark.read.option("inferSchema", True).option("header", "true") \
    .csv(f"s3a://raw/trips.txt").toPandas()
# show data
df_trips.head(10)

,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id
0,1012-10,USD,1012-10-0,Jd. Monte Belo,0,81072
1,1012-10,USD,1012-10-1,Term. Jd. Britania,1,81073
2,1012-21,U__,1012-21-0,Jd. Rosinha,0,81195
3,1014-10,U__,1014-10-0,Cptm Perus,0,82855
4,1015-10,USD,1015-10-0,Chác. Maria Trindade,0,81148
5,1016-10,USD,1016-10-0,Shop. Center Norte,0,72283
6,1016-10,USD,1016-10-1,Cem. Do Horto,1,82172
7,1017-10,USD,1017-10-0,Conexão Vl. Iório,0,72355
8,1017-10,USD,1017-10-1,Perus,1,72356
9,1018-10,USD,1018-10-0,Metrô Santana,0,71029


In [5]:
df_shapes = pd.read_csv('trips_stops.csv')
# show data
df_shapes.head(10)

,Unnamed: 0,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled,stop_id,stop_distance
0,0,81072,-23.432024,-46.787121,1,0.000000,NaN,NaN
1,1,81072,-23.431827,-46.787153,2,22.061337,301790.0,10.975496
2,2,81072,-23.431764,-46.787163,3,29.113285,NaN,NaN
3,3,81072,-23.431614,-46.787187,4,45.906006,NaN,NaN
4,4,81072,-23.431590,-46.787191,5,48.596806,NaN,NaN
5,5,81072,-23.431554,-46.787196,6,52.615639,NaN,NaN
6,6,81072,-23.431512,-46.787202,7,57.306896,NaN,NaN
7,7,81072,-23.431438,-46.787210,8,65.543129,NaN,NaN
8,8,81072,-23.431388,-46.787215,9,71.103470,NaN,NaN
9,9,81072,-23.431378,-46.787218,10,72.248848,NaN,NaN


In [15]:
import random


list_trips = df_trips['trip_id'].values

data = []

for s in df_shapes.values:
    data.append({"vs_ta" : 1, "trip_id":random.choice(list_trips),  'stop_id': float(s[6]), 'vs_p': '1', 'shape_id': int(s[1]) })
    
data[:10]
    

[{'vs_ta': 1,
  'trip_id': '407G-10-1',
  'stop_id': nan,
  'vs_p': '1',
  'shape_id': 81072},
 {'vs_ta': 1,
  'trip_id': '2435-10-0',
  'stop_id': 301790.0,
  'vs_p': '1',
  'shape_id': 81072},
 {'vs_ta': 1,
  'trip_id': '6028-10-0',
  'stop_id': nan,
  'vs_p': '1',
  'shape_id': 81072},
 {'vs_ta': 1,
  'trip_id': '148P-10-1',
  'stop_id': nan,
  'vs_p': '1',
  'shape_id': 81072},
 {'vs_ta': 1,
  'trip_id': '807A-10-1',
  'stop_id': nan,
  'vs_p': '1',
  'shape_id': 81072},
 {'vs_ta': 1,
  'trip_id': '809P-10-0',
  'stop_id': nan,
  'vs_p': '1',
  'shape_id': 81072},
 {'vs_ta': 1,
  'trip_id': '2720-31-1',
  'stop_id': nan,
  'vs_p': '1',
  'shape_id': 81072},
 {'vs_ta': 1,
  'trip_id': '372U-10-1',
  'stop_id': nan,
  'vs_p': '1',
  'shape_id': 81072},
 {'vs_ta': 1,
  'trip_id': '3134-10-1',
  'stop_id': nan,
  'vs_p': '1',
  'shape_id': 81072},
 {'vs_ta': 1,
  'trip_id': '8073-10-0',
  'stop_id': nan,
  'vs_p': '1',
  'shape_id': 81072}]

In [19]:
df_chegadas = spark.createDataFrame(data)
print('gravando Clickhouse')
df_chegadas.write \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", url) \
    .option("user", user) \
    .option("password", password) \
    .option("dbtable", "chegadas") \
    .option("createTableOptions", "ENGINE=Log()") \
    .mode("overwrite") \
    .save()
print('gravando Minio')
df_chegadas.write.csv(f"s3a://trusted/chegadas.csv")

gravando Clickhouse
gravando Minio
